<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Git Branches

## Branching and Merging Locally

### Create a Local Repo

Use the terminal to create a new local repository.

```bash
>> mkdir my-git-repo
>> cd my-git-repo
>> git init
Reinitialized existing Git repository in /Users/gGandenberger/ga/ent_vitol/lessons/git_branches/my-git-repo/.git/
```

Throw some text in a file.

```bash
>> cat > "quotes.txt" << EOF
>> heredoc> I am an egotistical bastard, and I name all my projects after myself. First Linux, now git.
>> heredoc>   -- Linus Torvalds
>> heredoc> EOF
```

> **Side note:** `> "quotes.txt"` redirects the output of `cat` into a file named "quotes.txt." `<< EOF` tells bash that the following lines are part of the text to pass to `cat` until a line that matches "EOF", which signals the end of the text. "EOF" stands for "end of file." We could have used something else here, such as "STOP," but EOF is conventional.

Commit our changes.

```bash
>> git add quotes.txt
>> git commit -m "Add Torvalds quote"
[master (root-commit) c7fc154] Add Torvalds quote
 1 file changed, 2 insertions(+)
 create mode 100644 quotes.txt
```

### Make Changes in a Branch

Until now, we have done all of our work on a single `master` branch. But when peole are depending on your code, it is a good idea to separate work in progress from the canonical, working code.

Create a new branch to hold our changes until we are ready to roll them out to users.

```bash
>> git branch lyrics
```

That command **created** the branch, but it didn't move us to it, as we can see when we run this command.

```bash
>> git branch
  lyrics
* master
```

We need to "check out" the new branch.

```bash
>> git checkout lyrics
Switched to branch 'lyrics'
```

```bash
>> git branch
* lyrics
  master
```

Throw more text in our quotes file. **The typo is intentional--don't correct it yet!**

```bash
>> cat >> "quotes.txt" << EOF
>> heredoc> There is no dark side of the moon really. As a matter of fact it is all drk.
>> heredoc>   -- Pink Floyd, Eclipse
>> heredoc> EOF
```

> **Side note:** `>> "quotes.txt"` **appends** to "quotes.txt", whereas `> "quotes.txt"` overwrites the current file contents.

Create a new commit on our `lyrics` branch.

```bash
>> git add quotes.txt
>> git commit -m "Add Pink Floyd quote."
[lyrics ff75c4d] Add Pink Floyd quote.
 1 file changed, 2 insertions(+)
```

Confirm that we have two commits in our git history.

```bash
>> git log
```

### Merge Changes

When we are happy with the changes in our `lyrics` branch, we can merge them into `master`.

First, we check out `master`.

```bash
>> git checkout master
Switched to branch 'master'
```

Confirm that we have only one commit in our git history on this branch.

```bash
>> git log
```

Merge `lyrics`.

```bash
>> git merge lyrics
Updating c7fc154..ff75c4d
Fast-forward
 quotes.txt | 2 ++
 1 file changed, 2 insertions(+)
```

Confirm that we have two commits in our git history.

```bash
>> git log
```

If we are done with `lyrics`, we can delete it.

```bash
>> git branch -d lyrics
Deleted branch lyrics (was ff75c4d).
```

### Resolve a Conflict

What happens if we want to merge into `master`, but `master` has changed in the meantime? 

Make and check out a new branch.

```bash
>> git branch zen
>> git checkout zen
Switched to branch 'zen'
```

Make some changes.

```bash
>> cat >> "quotes.txt" << EOF
>> heredoc> There should be one--and preferably only one--obvious way to do it.
>> heredoc>   -- Tim Peters
>> heredoc> EOF
```

```bash
>> git add quotes.txt
>> git commit -m "Add Zen of Python quote"
[zen 0cd35b0] Add Zen of Python quote
 1 file changed, 2 insertions(+)
```

Go back to `master`.

```bash
>> git checkout master
```

Make some changes.

```bash
>> cat >> "quotes.txt" << EOF
>> heredoc> I have no special talents. I am only passionately curious.
>> heredoc>   -- Albert Einstein
>> heredoc> EOF
```

```bash
>> git add quotes.txt
>> git commit -m "Add Einstein quote"
[master d5e4930] Add Einstein quote
 1 file changed, 2 insertions(+)
```

Try to merge the changes from `zen` branch.

```bash
>> git merge zen
Auto-merging quotes.txt
CONFLICT (content): Merge conflict in quotes.txt
Automatic merge failed; fix conflicts and then commit the result.
```

We now have two different versions of lines 5-6: the Zen of Python quote from our `zen` branch, and the Einstein quote that is already in master.

`git` has no way to know whether we want the version in `master`, the version in `zen`, or some combination of the two. Rather than guessing, it just tells us that there is a conflict and asks us to fix it and commit the results.

Let's use `nano` to resolve the conflict.

```bash
nano quotes.txt
```

`git` has added special annotations showing us where the conflicts are.

![](../assets/images/conflict.png)

If we want both quotes, then all we need to do to the file is to delete those annotations.

When you think you are done editing files with conflicts, it is a good practice to use this command to confirm that all conflict markers have been deleted.

```bash
>> git diff --check
```

Now we can finish the merge.

```bash
>> git add quotes.txt
>> git commit -m "Merge zen branch"
[master 6f0b3df] Merge zen branch
```

Our git history now contains a "merge commit" in addition to the separate commits from `master` and `zen`.

```bash
>> git log
```

If there are no conflicts, then `git` will do a "fast-forward merge" that does not generate a separate merge commit.

Cleanup step: delete `zen`.

```bash
>> git branch -d zen
Deleted branch zen (was e7575ea).
```

## Branching and Merging on GitHub

When multiple people are working on a codebase together, they might want to review each other's changes before merging them into `master`. GitHub supports this practice through "pull requests."

First, create `my-git-repo` on GitHub.

![](../assets/images/make_remote1.png)

![](../assets/images/make_remote2.png)

Set it up as our remote repository, and push the contents of our local repo to it.

```bash
git remote add origin <url>
git push -u origin master
```

Next, create an **issue** regarding the typo in our Pink Floyd quote. It is not necessary to create an issue first, but the issue is useful as documentation, and PRs very often address previously existing issues.

Click the "issues" tab:

![](../assets/images/make_issue1.png)

Click the "New Issue" button:

![](../assets/images/make_issue2.png)

Describe the issue, and submit:

![](../assets/images/make_issue3.png)

Now fix the issue. We can use a shortcut to create a branch and check it out in one step.

```bash
>> git checkout -b fix-typo
M	quotes.txt
Switched to a new branch 'fix-typo'
```

Use `sed` to do a find-and-replace operation:

```bash
>> sed -i "" "s/drk/dark/" quotes.txt
```

Confirm that our change happened as expected:

```bash
>> git diff
```

Commit our changes.

```bash
>> git add quotes.txt
>> git commit -m "Fix typo"
```

Push `fix-typo` to GitHub.

```bash
>> git push -u origin fix-typo
```

Go back to the main repo page on GitHub and click the button to open a PR.

![](../assets/images/open_pr1.png)

If we put "Closes #1" in the description of the PR, then merging the PR will automatically close our issue.

![](../assets/images/open_pr2.png)

Merge the pull request. (In this case we are both the submitter and the reviewer of the pull request. Usually those roles belong to different people.)

Typically at this point we would request a review from someone else, but in this case we can act as our own reviewer and just merge the changes.

![](../assets/images/merge_pr.png)

Delete the remote `fix-typo` branch.

![](../assets/images/delete_branch.png)

Back on your local machine, switch back to the `master` branch.

```bash
>> git checkout master
Switched to branch 'master'
Your branch is up to date with 'origin/master'.
```

Pull changes from `origin`

```bash
>> git pull
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 1 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (1/1), done.
From https://git.generalassemb.ly/greggandenberger/my-git-repo
   4aea7cb..a8ed19c  master     -> origin/master
Updating 4aea7cb..a8ed19c
Fast-forward
 quotes.txt | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
```

Delete the local `fix-typo` branch.

```bash
>> git branch -d fix-typo
Deleted branch fix-typo (was a3a46d7).
```

## Summary

- Git branches allow us to work on changes to our code without affecting the stable code that we depend on.
- Pull requests allow collaborators to review each other's changes before merging them into `master`.